## Using AI Services for Analyzing Public Data
by Manav Sehgal | on APR 30 2019

So far we have been working with structured data in flat files as our data source. What if the source is images and unstructured text. AWS AI services provide vision, transcription, translation, personalization, and forecasting capabilities without the need for training and deploying machine learning models. AWS manages the machine learning complexity, you just focus on the problem at hand and send required inputs for analysis and receive output from these services within your applications.

Extending our open data analytics use case to New York Traffic let us use the AWS AI services to turn open data available in social media, Wikipedia, and other sources into structured datasets and insights.

We will start by importing dependencies for AWS SDK, Python Data Frames, file operations, handeling JSON data, and display formatting. We will initialize the Rekognition client for use in the rest of this notebook.

In [216]:
import boto3
import pandas as pd
import io
import json
from IPython.display import display, Markdown, Image

rekognition = boto3.client('rekognition','us-east-1')
image_bucket = 'open-data-analytics-taxi-trips'

### Show Image
We will work with a number of images so we need a way to show these images within this notebook. Our function creates a public image URL based on S3 bucket and key as input.

In [217]:
def show_image(bucket, key, img_width = 500):
    # [TODO] Load non-public images
    return Image(url='https://s3.amazonaws.com/' + bucket + '/' + key, width=img_width)

In [218]:
show_image(image_bucket, 'images/traffic-in-manhattan.jpg', 1024)

### Image Labels
One of use cases for traffic analytics is processing traffic CCTV imagery or social media uploads. Let's consider a traffic location where depending on number of cars, trucks, and pedestrians we can identify if there is a traffic jam. This insight can be used to better manage flow of traffic around the location and plan ahead for future use of this route.

First step in this kind of analytics is to recognize that we are actually looking at an image which may represent a traffic jam. We create ``image_labels`` function which uses ``detect_lables`` Rekognition API to detect objects within an image. The function prints labels detected with confidence score.

In the given example notice somewhere in the middle of the labels listing at 73% confidence the Rekognition computer vision model has actually determined a traffic jam.

In [219]:
def image_labels(bucket, key):
    image_object = {'S3Object':{'Bucket': bucket,'Name': key}}

    response = rekognition.detect_labels(Image=image_object)
    for label in response['Labels']:
        print('{} ({:.0f}%)'.format(label['Name'], label['Confidence']))

In [220]:
image_labels(image_bucket, 'images/traffic-in-manhattan.jpg')

Vehicle (100%)
Automobile (100%)
Transportation (100%)
Car (100%)
Human (99%)
Person (99%)
Truck (98%)
Machine (96%)
Wheel (96%)
Clothing (87%)
Apparel (87%)
Footwear (87%)
Shoe (87%)
Road (75%)
Traffic Jam (73%)
City (73%)
Urban (73%)
Metropolis (73%)
Building (73%)
Town (73%)
Cab (71%)
Taxi (71%)
Traffic Light (68%)
Light (68%)
Neighborhood (62%)
People (62%)
Pedestrian (59%)


### Image Label Count
Now that we have a label detecting a traffic jam and some of the ingredients of a busy traffic location like pedestrians, trucks, cars, let us determine quantitative data for benchmarking different traffic locations. If we can count the number of cars, trucks, and persons in the image we can compare these numbers with other images. Our function does just that, it counts the number of instances of a matching label.

In [221]:
def image_label_count(bucket, key, match):    
    image_object = {'S3Object':{'Bucket': bucket,'Name': key}}

    response = rekognition.detect_labels(Image=image_object)
    count = 0
    for label in response['Labels']:
        if match in label['Name']:
            for instance in label['Instances']:
                count += 1
    print(f'Found {match} {count} times.')

In [222]:
image_label_count(image_bucket, 'images/traffic-in-manhattan.jpg', 'Car')

Found Car 9 times.


In [223]:
image_label_count(image_bucket, 'images/traffic-in-manhattan.jpg', 'Truck')

Found Truck 4 times.


In [224]:
image_label_count(image_bucket, 'images/traffic-in-manhattan.jpg', 'Person')

Found Person 8 times.


### Image Text
Another use case of traffic location analytics using social media content is to understand more about a traffic location and instance if there is an incident reported, like an accident, jam, or VIP movement. For a computer program to understand a random traffic location, it may help to capture any text within the image. The ``image_text`` function uses Amazon Rekognition service to detect text in an image.

You will notice that the text recognition is capable to read blurry text like "The Lion King", text which is at a perspective like the bus route, text which may be ignored by the human eye like the address below the shoes banner, and even the text representing the taxi number. Suddenly the image starts telling a story programmatically, about what time it may represent, what are the landmarks, which bus route, which taxi number was on streets, and so on.

In [225]:
def image_text(bucket, key, sort_column='', parents=True):
    response = rekognition.detect_text(Image={'S3Object':{'Bucket':bucket,'Name': key}})
    df = pd.read_json(io.StringIO(json.dumps(response['TextDetections'])))
    df['Width'] = df['Geometry'].apply(lambda x: x['BoundingBox']['Width'])
    df['Height'] = df['Geometry'].apply(lambda x: x['BoundingBox']['Height'])
    df['Left'] = df['Geometry'].apply(lambda x: x['BoundingBox']['Left'])
    df['Top'] = df['Geometry'].apply(lambda x: x['BoundingBox']['Top'])
    df = df.drop(columns=['Geometry'])
    if sort_column:
        df = df.sort_values([sort_column])
    if not parents:
        df = df[df['ParentId'] > 0]
    return df

In [226]:
show_image(image_bucket, 'images/nyc-taxi-signs.jpeg', 1024)

Sorting on ``Top`` column will keep the horizontal text together.

In [227]:
image_text(image_bucket, 'images/nyc-taxi-signs.jpeg', sort_column='Top', parents=False)

,Confidence,DetectedText,Id,ParentId,Type,Width,Height,Left,Top
14,91.874588,WAY,15,1.0,WORD,0.028470,0.019385,0.599400,0.109109
15,83.133957,6ASW,14,1.0,WORD,0.034089,0.018404,0.570143,0.126126
17,94.518997,HAN'S,17,2.0,WORD,0.070971,0.032111,0.388597,0.187187
16,99.643578,DELI,16,2.0,WORD,0.080892,0.041151,0.281320,0.201201
18,90.439888,&,18,3.0,WORD,0.027007,0.044044,0.364591,0.212212
19,99.936119,GROCERY,19,3.0,WORD,0.150999,0.042149,0.399850,0.217217
20,81.925537,ZiGi,20,4.0,WORD,0.027007,0.023035,0.595649,0.265265
21,95.180290,SHOES,21,5.0,WORD,0.041695,0.019078,0.621906,0.269269
22,91.584435,X29CONEYSL,23,5.0,WORD,0.108448,0.038509,0.887472,0.279279
23,90.353638,x29,24,5.0,WORD,0.038896,0.033245,0.888972,0.282282


### Detect Celebs
Traffic analytics may also involve detecting VIP movement to divert traffic or monitor security events. Detecting VIP in a scene starts with facial recognition. Our function ``detect_celebs`` works as well with political figures as it will with movie celebrities.

In [228]:
def detect_celebs(bucket, key, sort_column=''):
    image_object = {'S3Object':{'Bucket': bucket,'Name': key}}

    response = rekognition.recognize_celebrities(Image=image_object)
    df = pd.DataFrame(response['CelebrityFaces'])
    df['Width'] = df['Face'].apply(lambda x: x['BoundingBox']['Width'])
    df['Height'] = df['Face'].apply(lambda x: x['BoundingBox']['Height'])
    df['Left'] = df['Face'].apply(lambda x: x['BoundingBox']['Left'])
    df['Top'] = df['Face'].apply(lambda x: x['BoundingBox']['Top'])
    df = df.drop(columns=['Face'])
    if sort_column:
        df = df.sort_values([sort_column])
    return(df)

In [229]:
show_image(image_bucket, 'images/world-leaders.jpg', 1024)

In [230]:
detect_celebs(image_bucket, 'images/world-leaders.jpg', sort_column='Left')

,Id,MatchConfidence,Name,Urls,Width,Height,Left,Top
3,4Ev8IX1,100.0,Chulabhorn,[],0.020202,0.038973,0.015152,0.424905
5,3J795K,100.0,Manmohan Singh,[],0.018687,0.035171,0.131313,0.420152
25,f0JR5e,90.0,Mahinda Rajapaksa,[],0.016162,0.030418,0.145960,0.319392
30,3n7tl2O,88.0,Killah Priest,[www.imdb.com/name/nm0697334],0.014646,0.027567,0.162121,0.290875
12,2gC0Tc0e,100.0,Rosen Plevneliev,[],0.018182,0.034221,0.179293,0.367871
19,3LR2lb6j,56.0,Jerry Harrison,[],0.017172,0.032319,0.227273,0.330798
1,4hD40O,100.0,Thomas Boni Yayi,[],0.021717,0.040875,0.236364,0.399240
22,2F5LV4,63.0,Irwansyah,[www.imdb.com/name/nm2679097],0.016667,0.031369,0.274747,0.340304
8,3hk2qj5G,98.0,Cristina Fernández de Kirchner,[www.imdb.com/name/nm3231417],0.018687,0.035171,0.278283,0.414449
13,2sN1oC8s,100.0,Jorge Carlos Fonseca,[],0.018182,0.034221,0.280808,0.370722


### Comprehend Syntax
It is possible that many data sources represent natural language and free text. Understand structure and semantics from this unstructured text can help further our open data analytics use cases.

Let us assume we are processing traffic updates for structured data so we can take appropriate actions. First step in understanding natural language is to break it up into grammaticaly syntax. Nouns like "today" can tell about a particular event like when is the event occuring. Adjectives like "snowing" and "windy" tell what is happening at that moment in time. 

In [231]:
comprehend = boto3.client('comprehend', 'us-east-1')

traffic_update = """
It is snowing and windy today in New York. The temperature is 50 degrees Fahrenheit. 
The traffic is slow 10 mph with several jams along the I-86.
"""

In [232]:
def comprehend_syntax(text): 
    response = comprehend.detect_syntax(Text=text, LanguageCode='en')
    df = pd.read_json(io.StringIO(json.dumps(response['SyntaxTokens'])))
    df['Tag'] = df['PartOfSpeech'].apply(lambda x: x['Tag'])
    df['Score'] = df['PartOfSpeech'].apply(lambda x: x['Score'])
    df = df.drop(columns=['PartOfSpeech'])
    return df

In [233]:
comprehend_syntax(traffic_update)

,BeginOffset,EndOffset,Text,TokenId,Tag,Score
0,1,3,It,1,PRON,0.999971
1,4,6,is,2,VERB,0.557677
2,7,14,snowing,3,ADJ,0.687805
3,15,18,and,4,CONJ,0.999998
4,19,24,windy,5,ADJ,0.994336
5,25,30,today,6,NOUN,0.999980
6,31,33,in,7,ADP,0.999924
7,34,37,New,8,PROPN,0.999351
8,38,42,York,9,PROPN,0.998399
9,42,43,.,10,PUNCT,0.999998


### Comprehend Entities
More insights can be derived by doing entity extraction from the natural langauage. These entities can be date, location, quantity, among others. Just few of the entities can tell a structured story to a program.

In [234]:
def comprehend_entities(text):
    response = comprehend.detect_entities(Text=text, LanguageCode='en')
    df = pd.read_json(io.StringIO(json.dumps(response['Entities'])))
    return df

In [235]:
comprehend_entities(traffic_update)

,BeginOffset,EndOffset,Score,Text,Type
0,25,30,0.839589,today,DATE
1,34,42,0.998423,New York,LOCATION
2,63,84,0.984396,50 degrees Fahrenheit,QUANTITY
3,107,113,0.992498,10 mph,QUANTITY
4,142,146,0.990993,I-86,LOCATION


### Comprehend Phrases
Analysis of phrases within narutal language text complements the other two methods for a program to better route the actions based on derived structure of the event.

In [236]:
def comprehend_phrases(text):
    response = comprehend.detect_key_phrases(Text=text, LanguageCode='en')
    df = pd.read_json(io.StringIO(json.dumps(response['KeyPhrases'])))
    return df

In [237]:
comprehend_phrases(traffic_update)

,BeginOffset,EndOffset,Score,Text
0,25,30,0.988285,today
1,34,42,0.997397,New York
2,44,59,0.999752,The temperature
3,63,73,0.789843,50 degrees
4,87,98,0.999843,The traffic
5,107,113,0.924737,10 mph
6,119,131,0.998428,several jams
7,138,146,0.997108,the I-86


### Comprehend Sentiment
Sentiment analysis is common for social media user generated content. Sentiment can give us signals on the users' mood when publishing such social data.

In [238]:
def comprehend_sentiment(text):
    response = comprehend.detect_sentiment(Text=text, LanguageCode='en')
    return response['SentimentScore']

In [239]:
comprehend_sentiment(traffic_update)

{'Positive': 0.04090394824743271,
 'Negative': 0.3745909333229065,
 'Neutral': 0.5641733407974243,
 'Mixed': 0.020331736654043198}

### Change Log

This section captures changes and updates to this notebook across releases.

#### Usability and sorting for text and face detection - Release 3 MAY 2019
Functions ``image_text`` and ``detect_celeb`` can now sort results based on a column name. Function ``image_text`` can optionally show results without parent-child relations.

Usability update for ``comprehend_syntax`` function to split ``part of speech`` dictionary value into separate Tag and Score columns.

#### Launch - Release 30 APR 2019
This is the launch release which builds the AWS Open Data Analytics API for using AWS AI services to analyze public data.


---
#### Using AI Services for Analyzing Public Data
by Manav Sehgal | on APR 30 2019
